In [132]:
import pandas as pd
from pandasql import sqldf

query = lambda q: sqldf(q, globals())

caixas = pd.read_csv('../data/caixas.csv')
estoque = pd.read_csv('../data/estoque.csv')

In [133]:
cmd = f"""
    SELECT DISTINCT 
        caixas.CAIXA_ID, 
        MIN(estoque.ANDAR) AS ANDAR, 
        estoque.SKU,
        SUM(caixas.PECAS) AS PECAS
    FROM caixas
    JOIN estoque
        ON caixas.SKU = estoque.SKU
        AND caixas.PECAS <= estoque.PECAS
    GROUP BY caixas.CAIXA_ID, estoque.SKU"""
caixas_andares = query(cmd)
print(caixas_andares)

       CAIXA_ID  ANDAR        SKU  PECAS
0            12      1      SKU_1     10
1            12      2     SKU_10      6
2            12      0     SKU_11      3
3            12      0     SKU_12      4
4            12      2     SKU_13     18
...         ...    ...        ...    ...
22065      6938      0  SKU_12898      2
22066      6939      0  SKU_12756     10
22067      6939      0  SKU_12898      3
22068      6940      0  SKU_12755      1
22069      6940      0  SKU_12756     10

[22070 rows x 4 columns]


In [134]:
cmd = f"""
    SELECT COUNT(DISTINCT CAIXA_ID) AS TOTAL_CAIXAS
    FROM caixas"""
print(query(cmd))

for qtd_andares in range(1, 4):
    print(f'\nPrecisam de {qtd_andares} {("andares" if qtd_andares > 1 else "andar")}:')
    cmd = f"""
    SELECT COUNT(DISTINCT CAIXA_ID) AS QTD_CAIXAS
    FROM caixas_andares
    WHERE CAIXA_ID IN (
        SELECT CAIXA_ID 
        FROM caixas_andares
        GROUP BY CAIXA_ID
        HAVING COUNT(DISTINCT ANDAR) = {qtd_andares})"""
    print(query(cmd))

   TOTAL_CAIXAS
0          2072

Precisam de 1 andar:
   QTD_CAIXAS
0        1277

Precisam de 2 andares:
   QTD_CAIXAS
0         354

Precisam de 3 andares:
   QTD_CAIXAS
0         441


In [141]:
cmd = f"""
    SELECT CAIXA_ID, COUNT(SKU) AS SKUS, SUM(PECAS) AS PECAS
    FROM caixas_andares
    GROUP BY CAIXA_ID""" 
caixas_pecas = query(cmd)
print(caixas_pecas)

      CAIXA_ID  SKUS  PECAS
0           12    17    238
1           13    31    333
2           45    27    229
3           46    26    200
4           47    17    221
...        ...   ...    ...
2067      6936     4    119
2068      6937     7     91
2069      6938     2      8
2070      6939     2     13
2071      6940     2     11

[2072 rows x 3 columns]


o parâmetro de priorização usado para as caixas foi andar mais baixo e menor contagem total de peças

In [145]:
cmd = f"""
    SELECT caixas_pecas.CAIXA_ID, caixas_andares.ANDAR, caixas_andares.SKU, caixas_andares.PECAS
    FROM caixas_andares
    JOIN caixas_pecas
        ON caixas_andares.CAIXA_ID = caixas_pecas.CAIXA_ID
    ORDER BY caixas_andares.ANDAR ASC, caixas_pecas.PECAS ASC""" # parâmetro de priorização de caixas
caixas_andares = query(cmd)
print(caixas_andares)

       CAIXA_ID  ANDAR        SKU  PECAS
0           612      0  SKU_19448      1
1           613      0  SKU_19448      1
2           614      0  SKU_19448      1
3           616      0  SKU_19448      1
4           617      0  SKU_19448      1
...         ...    ...        ...    ...
22065      4166      2   SKU_2154      4
22066      2491      2  SKU_17635      8
22067       704      2    SKU_144      4
22068       704      2    SKU_244      6
22069       704      2    SKU_288      2

[22070 rows x 4 columns]


In [ ]:
caixas_real = caixas.copy()
estoque_real = estoque.copy()

for caixa in caixas_pecas['CAIXA_ID']:
    print(f'\nCaixa {caixa} no andar {andar}')